In [6]:
import sys
import os

thesis_path = "/" + os.path.join(
    *os.getcwd().split(os.path.sep)[:-1])
sys.path.append(thesis_path)


In [7]:
from evaluations.performance_01 import get_per_sample_metrics
metrics_df = get_per_sample_metrics()

In [10]:
agg_df = metrics_df.pivot_table(
        index=["seed", "model", "checkpoint", "dataset"],
        values=["tp", "fp", "fn"],
        aggfunc="sum").reset_index()
agg_df["precision"] = 100 * agg_df["tp"] / (agg_df["tp"] + agg_df["fp"])
agg_df["recall"] = 100 * agg_df["tp"] / (agg_df["tp"] + agg_df["fn"])
agg_df["f1"] = 2 * agg_df["precision"] * agg_df["recall"] / (
    agg_df["precision"] + agg_df["recall"])

agg_df.iloc[agg_df[(agg_df["model"]=="t5_asp") & (agg_df["dataset"]=="lowner_test")]["f1"].idxmax()]

seed                    1
model              t5_asp
checkpoint           last
dataset       lowner_test
fn                   6977
fp                   5354
tp                  36739
precision       87.280545
recall          84.040168
f1              85.629713
Name: 4, dtype: object

In [3]:
metrics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10897200 entries, 0 to 10897199
Data columns (total 15 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   seed         object
 1   model        object
 2   checkpoint   object
 3   dataset      object
 4   doc_id       object
 5   targets      int64 
 6   entity_type  object
 7   tp           int64 
 8   fn           int64 
 9   fp           int64 
 10  error_type1  int64 
 11  error_type2  int64 
 12  error_type3  int64 
 13  error_type4  int64 
 14  error_type5  int64 
dtypes: int64(9), object(6)
memory usage: 1.2+ GB


In [4]:
metrics_df.describe()

,targets,tp,fn,fp,error_type1,error_type2,error_type3,error_type4,error_type5
count,1.089720e+07,1.089720e+07,1.089720e+07,1.089720e+07,1.089720e+07,1.089720e+07,1.089720e+07,1.089720e+07,1.089720e+07
mean,2.161282e-01,1.905645e-01,2.556363e-02,1.679248e-02,5.740924e-03,1.422549e-02,4.029292e-03,1.301343e-03,5.957035e-03
std,5.983969e-01,5.668767e-01,1.740430e-01,1.412545e-01,8.191047e-02,1.271610e-01,6.762273e-02,3.704992e-02,8.482927e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.200000e+01,1.200000e+01,8.000000e+00,7.000000e+00,7.000000e+00,8.000000e+00,6.000000e+00,3.000000e+00,5.000000e+00


# Model performance
## Macro-F1 + 5 Error Types avg+std

In [5]:
# Macro - F1
from evaluations.performance_01 import get_f1

get_f1(metrics_df, "last", "lowner_test")

model checkpoint      dataset         f1          
                                                        mean       std
4                  t5_asp       last  lowner_test  85.481521  0.163211
10             t5_asp_gaz       last  lowner_test  87.308047  0.018759
16        t5_asp_gaz_sent       last  lowner_test  88.132432  0.093228
22       t5_asp_lownergaz       last  lowner_test  87.478254  2.444407
28  t5_asp_lownergaz_sent       last  lowner_test  89.600103  0.029477
34            t5_asp_sent       last  lowner_test  86.724683  0.218033

In [6]:
# 5 Error types
from evaluations.performance_01 import get_error_types
get_error_types(metrics_df, "last", "lowner_test")

model checkpoint      dataset  error_type1              \
                                                          mean         std   
4                  t5_asp       last  lowner_test  1652.666667   71.514567   
10             t5_asp_gaz       last  lowner_test  1610.333333   33.231511   
16        t5_asp_gaz_sent       last  lowner_test  1699.666667   16.010413   
22       t5_asp_lownergaz       last  lowner_test  1519.000000  148.704405   
28  t5_asp_lownergaz_sent       last  lowner_test  1577.000000  111.139552   
34            t5_asp_sent       last  lowner_test  1473.000000   85.281886   

    error_type2               error_type3            error_type4             \
           mean          std         mean        std        mean        std   
4   3078.000000    60.307545  1595.000000   4.582576  492.000000  16.370706   
10  3083.666667    36.828431  1102.000000  26.888659  379.000000  11.000000   
16  2618.333333    20.744477  1006.000000   4.358899  374.333333   5.859465   
22  4099.000000  2043.395948   908.666667  27.970222  242.666667  13.051181   
28  2622.333333   121.705930   855.666667   6.506407  234.333333   1.527525   
34  3334.000000   134.357731  1235.000000  16.822604  402.000000   7.000000   

    error_type5             
           mean        std  
4   1840.666667  56.518434  
10  1662.666667   5.507571  
16  1610.000000  27.622455  
22  1321.333333  26.312228  
28  1277.666667  38.083242  
34  1655.333333  12.858201

## Macro F1 + errors by entity type

In [3]:
from evaluations.performance_01 import get_f1_per_entity

get_f1_per_entity(metrics_df, "last", "lowner_test")

entity_type                  model checkpoint      dataset corporation  \
                                                                  mean   
                                                                    f1   
4                           t5_asp       last  lowner_test   75.113433   
10                      t5_asp_gaz       last  lowner_test   81.463768   
16                 t5_asp_gaz_sent       last  lowner_test   83.503061   
22                t5_asp_lownergaz       last  lowner_test   81.709624   
28           t5_asp_lownergaz_sent       last  lowner_test   84.335537   
34                     t5_asp_sent       last  lowner_test   79.699732   

entity_type           creative-work                group             location  \
                  std          mean       std       mean       std       mean   
                   f1            f1        f1         f1        f1         f1   
4            0.462709     71.650596  0.102965  79.986712  0.173166  87.925792   
10           0.390062     75.532770  0.055493  84.282475  0.440974  88.519457   
16           0.065501     76.397745  0.465148  85.533325  0.108788  89.173644   
22           1.615418     77.102966  3.456147  85.925062  1.160642  89.815369   
28           0.382686     80.932509  0.187027  87.375040  0.112857  90.412207   
34           0.487246     74.173166  0.369699  82.780910  0.455662  88.244192   

entity_type               person              product            
                  std       mean       std       mean       std  
                   f1         f1        f1         f1        f1  
4            0.274731  94.275321  0.030955  44.775935  0.521565  
10           0.213985  94.758308  0.056385  51.398970  0.554407  
16           0.070792  95.064632  0.065441  60.882828  0.576907  
22           0.153259  92.551586  5.597763  50.420167  2.201350  
28           0.064019  95.841276  0.124411  59.474805  0.588127  
34           0.154587  94.552950  0.113219  53.592878  0.824031

In [3]:
from evaluations.performance_01 import get_error_types_per_entity

get_error_types_per_entity(metrics_df, "last", "lowner_test")

entity_type                  model checkpoint      dataset corporation  \
                                                                  mean   
                                                           error_type1   
4                           t5_asp       last  lowner_test   75.333333   
10                      t5_asp_gaz       last  lowner_test   98.666667   
16                 t5_asp_gaz_sent       last  lowner_test   88.666667   
22                t5_asp_lownergaz       last  lowner_test   81.000000   
28           t5_asp_lownergaz_sent       last  lowner_test   87.000000   
34                     t5_asp_sent       last  lowner_test   76.000000   

entity_type             creative-work                   group              \
                    std          mean         std        mean         std   
            error_type1   error_type1 error_type1 error_type1 error_type1   
4              4.725816    415.000000   15.132746  233.000000    8.660254   
10             4.163332    414.333333    4.041452  205.333333   25.501634   
16             6.658328    458.000000   24.515301  208.666667    9.237604   
22             7.211103    377.666667   53.425961  185.666667   21.594752   
28             2.645751    385.333333   15.567059  186.666667   23.158872   
34             3.464102    396.666667   30.353473  191.333333   20.599353   

entity_type    location  ... creative-work                   group  \
                   mean  ...          mean         std        mean   
            error_type1  ...   error_type5 error_type5 error_type5   
4            375.666667  ...    326.666667   11.676187  304.333333   
10           356.666667  ...    258.666667   10.016653  258.666667   
16           387.000000  ...    273.666667   18.770544  243.666667   
22           359.000000  ...    198.000000   13.453624  211.333333   
28           369.666667  ...    182.333333   16.921387  200.000000   
34           315.333333  ...    269.000000   10.000000  277.666667   

entity_type                location                  person              \
                    std        mean         std        mean         std   
            error_type5 error_type5 error_type5 error_type5 error_type5   
4              8.020806  937.000000   45.574115  173.666667    2.886751   
10            22.479620  892.333333   28.746014  172.333333    8.082904   
16             4.163332  843.000000    2.000000  166.666667   14.011900   
22            12.858201  700.333333   17.616280  128.000000    7.810250   
28             5.291503  682.333333   11.930353  129.000000    7.549834   
34             5.131601  855.666667   18.475209  169.000000    8.544004   

entity_type     product              
                   mean         std  
            error_type5 error_type5  
4             43.000000    5.000000  
10            28.000000    4.000000  
16            31.333333    0.577350  
22            31.666667    0.577350  
28            32.666667    5.859465  
34            32.333333    2.081666  

[6 rows x 63 columns]

## EECR + ECR for labeled data + search results

In [3]:
# EECR - labeled data
from evaluations.performance_01 import get_labeled_data_eecr

get_labeled_data_eecr()


In [3]:
# EECR - search results

#def get_search_results_data():
from evaluations.performance_01 import get_search_results_data

search_results, dataset = get_search_results_data()



In [5]:
search_results["t5_asp_gaz"]["lowner_test"]

{0: [{'content': 'the trial',
   'content_type': 'text',
   'score': 0.8858113525853097,
   'meta': {'data_type': 'gazetteers', 'type': 'creative-work'},
   'id_hash_keys': ['content'],
   'embedding': None,
   'id': '3f3ba1e6-eeb7-4bcd-85f1-c3f606223df5'},
  {'content': 'the trial of the incredible hulk',
   'content_type': 'text',
   'score': 0.8696095180024672,
   'meta': {'data_type': 'gazetteers', 'type': 'creative-work'},
   'id_hash_keys': ['content'],
   'embedding': None,
   'id': '8a6c31b1-98f6-400f-92fe-1defd48d7d57'},
  {'content': 'the trial of christine keeler',
   'content_type': 'text',
   'score': 0.8627445521568003,
   'meta': {'data_type': 'gazetteers', 'type': 'creative-work'},
   'id_hash_keys': ['content'],
   'embedding': None,
   'id': 'f0e66cbd-5ed5-411f-acc9-fe5e7eade761'},
  {'content': 'the tokyo trial',
   'content_type': 'text',
   'score': 0.8493110867361855,
   'meta': {'data_type': 'gazetteers', 'type': 'creative-work'},
   'id_hash_keys': ['content'],
